<a href="https://colab.research.google.com/github/sachinsaraff10/NFL-tracker/blob/main/nfl_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

nfl_big_data_bowl_2025_path = kagglehub.competition_download('nfl-big-data-bowl-2025')
sachinsaraff_qbadvanced_path = kagglehub.dataset_download('sachinsaraff/qbadvanced')
sachinsaraff_2022qbstats_path = kagglehub.dataset_download('sachinsaraff/2022qbstats')
sachinsaraff_receiver_stats_path = kagglehub.dataset_download('sachinsaraff/receiver-stats')
sachinsaraff_more_qbstats_path = kagglehub.dataset_download('sachinsaraff/more-qbstats')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

qbstats=pd.read_excel('/kaggle/input/qbadvanced/qb stats.xlsx',header=1)

/kaggle/input/nfl-big-data-bowl-2025/players.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_7.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_9.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_6.csv
/kaggle/input/nfl-big-data-bowl-2025/games.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_8.csv
/kaggle/input/nfl-big-data-bowl-2025/player_play.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_4.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_3.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_5.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_1.csv
/kaggle/input/nfl-big-data-bowl-2025/plays.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_2.csv
/kaggle/input/2022qbstats/qbstats2022.xlsx
/kaggle/input/more-qbstats/FantasyPros_Fantasy_Football_Advanced_Stats_Report_QB.csv
/kaggle/input/receiver-stats/receiverstats.xlsx
/kaggle/input/qbadvanced/qb stats.xlsx


In [ ]:
qbstats.rename(columns={"Unnamed: 0":"Name"},inplace=True)

In [ ]:
qbstats.dropna(inplace=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as patches

# Sample player data: (x, y positions, orientation angle in degrees)
players = [
    (30, 20, 45),   # (x, y, angle)
    (50, 50, 90),
    (70, 30, 135),
]

circle_radius = 2  # Circle size
arrow_length = circle_radius * 1.5  # Adjust as needed
arrow_width = 0.4  # Thin arrow shaft

fig, ax = plt.subplots(figsize=(8, 5))

for x, y, angle in players:
    theta = np.radians(angle)  # Convert angle to radians

    # Compute arrow start position (from center to the edge of the circle)
    x_arrow_start = x + circle_radius * np.cos(theta)
    y_arrow_start = y + circle_radius * np.sin(theta)

    # Create and add circle (player body)
    circle = plt.Circle((x, y), circle_radius, color='blue', ec='black', lw=1, zorder=3)
    ax.add_patch(circle)

    # Create and add arrow (tail)
    arrow = patches.FancyArrow(
        x_arrow_start, y_arrow_start,  # Start at circle's edge
        arrow_length * np.cos(theta), arrow_length * np.sin(theta),  # Extend outward
        width=arrow_width, head_width=0, head_length=0, color='blue', zorder=2
    )
    ax.add_patch(arrow)

# Set field dimensions
ax.set_xlim(0, 100)
ax.set_ylim(0, 60)
ax.set_aspect('equal')
ax.set_title("Players with Integrated Arrow Movement")
ax.grid(True, linestyle="--", alpha=0.5)

plt.show()


In [ ]:
# Load CSV in chunks and save as Parquet
chunk_size = 1_000_000  # Adjust based on memory capacity
path = '/kaggle/input/nfl-big-data-bowl-2025'
csv_file = path + '/tracking_week_1.csv'

In [ ]:

parquet_file = 'play_tracking_data.parquet'


for i, chunk in enumerate(reader):
    chunk.to_parquet(f'{parquet_file}.part{i}', index=False, engine='pyarrow')

In [ ]:
reader = pd.read_csv(csv_file, chunksize=chunk_size)

chunks = []
for chunk in reader:
    chunks.append(chunk)  # Save each chunk

week_1 = pd.concat(chunks, ignore_index=True)

In [ ]:
plays = pd.read_csv(path + '/plays.csv')

In [ ]:
players= pd.read_csv(path+'/players.csv')

In [ ]:
player_play = pd.read_csv(path+'/player_play.csv')

In [ ]:
qb2022 = pd.read_excel('/kaggle/input/2022qbstats/qbstats2022.xlsx')

In [ ]:
qb2022.drop(columns="Unnamed: 1",inplace=True)

In [ ]:
week_1 = week_1.merge(
    players.loc[:,['position','nflId','height']],
    on = 'nflId',
    how='left',
)

In [ ]:
filteredframes=week_1.query('event=="pass_forward" & displayName == "football"').frameId
previousframes = filteredframes-1

In [ ]:
week_1['position'].fillna('football',inplace=True)

<ipython-input-6-b5d80c0627e0>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  week_1['position'].fillna('football',inplace=True)


In [ ]:
filtered_df = week_1.query('frameType == "AFTER_SNAP" & (position == "QB" | position == "football")')
# grouped_df = filtered_df.sort_values(by=["frameId", "position"])

In [ ]:
# Sort by gameId, playId, frameId, and position to ensure grouping
grouped_df = filtered_df.sort_values(by=["gameId", "playId", "frameId", "position"])

# Optional: Reset index for continuity
grouped_df = grouped_df.reset_index(drop=True)

In [ ]:
# Perform the pivot
pivoted_df = grouped_df.pivot_table(
    index=["gameId", "playId", "frameId"],
    columns="position",
    values=["x", "y"]
).reset_index()

# Flatten the column names
pivoted_df.columns = ['_'.join(col).strip('_') for col in pivoted_df.columns.to_flat_index()]

# Resulting columns: ["gameId", "playId", "frameId", "x_QB", "y_QB", "x_football", "y_football"]


In [ ]:
pivoted_df["x_diff"] = pivoted_df["x_football"] - pivoted_df["x_QB"]
pivoted_df["y_diff"] = pivoted_df["y_football"] - pivoted_df["y_QB"]

In [ ]:
pivoted_df["distance"] = np.sqrt(pivoted_df["x_diff"]**2 + pivoted_df["y_diff"]**2)

In [ ]:
pivoted_df = pivoted_df.round(2)

In [ ]:
import matplotlib.pyplot as plt

# Parameters
qb_x, qb_y = 50, 25       # QB position (example)
angle = np.radians(45)    # QB orientation (in degrees, converted to radians)
spread = np.radians(30)   # Cone spread angle (±30 degrees)
max_range = 50            # Maximum pass range (yards)

# Calculate cone edges
left_edge_angle = angle + spread
right_edge_angle = angle - spread

# Define edge points
def edge_point(origin_x, origin_y, angle, distance):
    x = origin_x + distance * np.cos(angle)
    y = origin_y + distance * np.sin(angle)
    return x, y

left_x, left_y = edge_point(qb_x, qb_y, left_edge_angle, max_range)
right_x, right_y = edge_point(qb_x, qb_y, right_edge_angle, max_range)

# Plot the cone
plt.figure(figsize=(10, 6))
plt.plot([qb_x, left_x], [qb_y, left_y], 'r-', label="Left Edge")
plt.plot([qb_x, right_x], [qb_y, right_y], 'b-', label="Right Edge")
plt.fill([qb_x, left_x, right_x], [qb_y, left_y, right_y], color='orange', alpha=0.3, label="Target Area")
plt.scatter(qb_x, qb_y, c='k', label="QB Position", zorder=5)

# Field boundaries
plt.xlim(0, 100)
plt.ylim(0, 53.3)
plt.gca().set_aspect('equal', adjustable='box')
plt.title("QB Pass Target Area")
plt.xlabel("Field X (yards)")
plt.ylabel("Field Y (yards)")
plt.legend()
plt.grid()
plt.show()


In [ ]:
week_1[['x','y']]=week_1[['x','y']].round(2)

In [ ]:
unique_combos = week_1[['gameId','playId']].drop_duplicates()

In [ ]:
import pickle

In [ ]:
save_directory = '/kaggle/working'

In [ ]:
with open("week_1.pkl", "wb") as f:
    pickle.dump(week_1, f)


In [ ]:
os.makedirs(save_directory, exist_ok=True)

In [ ]:
# Create a copy of globals() to iterate safely
global_vars = globals().copy()

# Identify and save all DataFrame variables
for var_name, var_value in global_vars.items():
    if isinstance(var_value, pd.DataFrame):
        file_path = os.path.join(save_directory, f"{var_name}.pkl")
        with open(file_path, 'wb') as f:
            pickle.dump(var_value, f)
        print(f"Saved {var_name} to {file_path}")


In [ ]:
def save_dfs(global_vars, save_directory):
    # enter global_vars as globals.copy() when running this
    for var_name, var_value in global_vars.items():
        if isinstance(var_value, pd.DataFrame):
            file_path = os.path.join(save_directory, f"{var_name}.pkl")
            with open(file_path, 'wb') as f:
                pickle.dump(var_value, f)
            print(f"Saved {var_name} to {file_path}")

In [ ]:
def load_dfs():
    dataframes = {}
    for filename in os.listdir("/kaggle/working"):
        if filename.endswith(".pkl"):
            try:
                file_path = os.path.join("/kaggle/working", filename)
                if os.path.getsize(file_path) == 0:
                    print(f"Skipping empty file: {filename}")
                    continue
                with open(file_path, "rb") as f:
                    var_name = filename.replace(".pkl", "")  # Extract variable name from filename
                    dataframes[var_name] = pickle.load(f)
            except (pickle.UnpicklingError, EOFError) as e:
                print(f"Skipping corrupted or incompatible file: {filename} - {e}")
    return dataframes

In [ ]:
dfs=load_dfs()

Skipping corrupted or incompatible file: _15.pkl - pickle data was truncated
Skipping empty file: more_qbstats.pkl


In [ ]:
dfs.keys()

dict_keys(['_3', '_30', 'players', 'qb_stats', '_13', 'pivoted_df', '_31', '_35', '_12', 'player_play', '_7', 'chunk', 'more_qb_adv', 'airtimes', '_4', 'qb2022', 'summary_df', '_32', 'eligible_receivers', 'temp', '_11', 'filtered_sample', '_21', 'unique_combos', 'filtered_df', '_25', '_16', 'qbstats', 'plays', 'grouped_df', 'receivers', '_28', '_29', 'week_1', 'plays_1', '_22', 'pass_speeds', '_2', '_27', '_23', 'sample_1', 'whole_sample', '_9', '_24'])

In [ ]:
dfs = []
for filename in os.listdir("/kaggle/working"):
    if filename.endswith(".pkl"):
        try:
            with open(os.path.join("/kaggle/working", filename), "rb") as f:
                data = pickle.load(f)
                dfs.append(data)
                print(f"Successfully loaded {filename}")
        except pickle.UnpicklingError as e:
            print(f"Failed to load {filename}: {e}")

Failed to load _15.pkl: pickle data was truncated
Successfully loaded players.pkl
Successfully loaded receivers.pkl
Successfully loaded sample_1.pkl
Successfully loaded plays.pkl
Successfully loaded _11.pkl
Successfully loaded eligible_receivers.pkl
Successfully loaded temp.pkl
Successfully loaded pass_speeds.pkl
Successfully loaded _30.pkl
Successfully loaded _27.pkl


EOFError: Ran out of input

In [ ]:
week_1=dfs['week_1']

In [ ]:
whole_sample=dfs['whole_sample']

In [ ]:
sample_1=week_1.query('frameType=="AFTER_SNAP"').sample(n=1)

sample_game = sample_1.iloc[0].gameId
sample_play = sample_1.iloc[0].playId
sample_frame = sample_1.iloc[0].frameId

whole_sample=week_1.query('gameId==@sample_game & playId==@sample_play & frameId== @sample_frame')

sample_1.iloc[0]

gameId                      2022091100
playId                            3267
nflId                          44851.0
displayName                Marcus Maye
frameId                            108
frameType                   AFTER_SNAP
time             2022-09-11 19:35:15.9
jerseyNumber                       6.0
club                                NO
playDirection                     left
x                                 70.9
y                                31.43
s                                 2.35
a                                 2.32
dis                               0.22
o                               146.01
dir                             279.27
event                              NaN
position                            FS
height                             6-0
Name: 6584610, dtype: object

In [ ]:
save_dfs(globals().copy(),'/kaggle/working')

Saved qbstats to /kaggle/working/qbstats.pkl
Saved _3 to /kaggle/working/_3.pkl
Saved week_1 to /kaggle/working/week_1.pkl
Saved qb_stats to /kaggle/working/qb_stats.pkl
Saved plays_1 to /kaggle/working/plays_1.pkl
Saved whole_sample to /kaggle/working/whole_sample.pkl
Saved sample_1 to /kaggle/working/sample_1.pkl
Saved _12 to /kaggle/working/_12.pkl
Saved pass_speeds to /kaggle/working/pass_speeds.pkl
Saved _16 to /kaggle/working/_16.pkl
Saved summary_df to /kaggle/working/summary_df.pkl
Saved _21 to /kaggle/working/_21.pkl
Saved _23 to /kaggle/working/_23.pkl
Saved _25 to /kaggle/working/_25.pkl
Saved _27 to /kaggle/working/_27.pkl
Saved _30 to /kaggle/working/_30.pkl
Saved _31 to /kaggle/working/_31.pkl
Saved _32 to /kaggle/working/_32.pkl
Saved airtimes to /kaggle/working/airtimes.pkl
Saved _35 to /kaggle/working/_35.pkl
Saved more_qb_adv to /kaggle/working/more_qb_adv.pkl
Saved qb2022 to /kaggle/working/qb2022.pkl
Saved _7 to /kaggle/working/_7.pkl


In [ ]:
save_dfs(globals().copy(),'/kaggle/working')

In [ ]:
qb_stats=dfs['qb2022']

In [ ]:

def eligible_receivers(qb_x, qb_y, angle, spread, max_range, receivers):
    """
    Identify receivers within the quarterback's triangular field of view.

    Parameters:
    - qb_x, qb_y: Quarterback's position (x, y).
    - angle: QB orientation (radians).
    - spread: Field of view spread angle (radians).
    - max_range: Maximum range of the triangle (yards).
    - receivers: List of tuples [(rx1, ry1), (rx2, ry2), ...] representing receiver positions.

    Returns:
    - List of indices of eligible receivers.
    """
    # Calculate the edges of the triangle
    left_edge_angle = angle + spread
    right_edge_angle = angle - spread

    # Define triangle vertices
    left_x, left_y = edge_point(qb_x, qb_y, left_edge_angle, max_range)
    right_x, right_y = edge_point(qb_x, qb_y, right_edge_angle, max_range)
    print(left_x,right_x,left_y,right_y)
    v1, v2, v3 = (qb_x, qb_y), (left_x, left_y), (right_x, right_y)
    # print(v1,v2,v3)

    # Check each receiver
    eligible = []
    for i, row in receivers.iterrows():
        rx, ry = row['x'], row['y']

        if is_point_in_triangle(rx, ry, v1, v2, v3):

            eligible.append(i)

    return eligible

def edge_point(origin_x, origin_y, angle, distance):
    """
    Compute the endpoint of a line segment given origin, angle, and distance.
    """
    x = origin_x + distance * np.cos(angle)
    y = origin_y + distance * np.sin(angle)
    x,y = x.astype(int),y.astype(int)
    return x, y

def is_point_in_triangle(px, py, v1, v2, v3):
    """
    Check if the point (px, py) lies within the triangle defined by vertices v1, v2, v3.
    Uses the cross-product method.
    """
    def sign(p1, p2, p3):
        """Helper function to calculate the sign of the cross product."""
        return (p1[0] - p3[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p3[1])

    # Calculate cross products for the triangle's edges
    sign1 = sign((px, py), v1, v2)
    sign2 = sign((px, py), v2, v3)
    sign3 = sign((px, py), v3, v1)
    # sign1,sign2,sign3  = sign1.astype(int),sign2.astype(int),sign3.astype(int)
    sign1,sign2,sign3 = round(sign1.item(),2),round(sign2.item(),2),round(sign3.item(),2)
    # print(sign1,sign2,sign3)
    # If all signs are non-negative or non-positive, the point is inside the triangle
    return ((sign1 >= 0) & (sign2 >= 0) & (sign3 >= 0)) | ((sign1 <= 0) & (sign2 <= 0) & (sign3 <= 0))


In [ ]:
edge_point(0,0,10,10)

(-8, -5)

In [ ]:
qb_x = whole_sample.query('position=="QB"').x
qb_y = whole_sample.query('position=="QB"').y
max_range = 50
qb_orientation= whole_sample.query('position=="QB"').o

In [ ]:
plays_1 = dfs['plays']

In [ ]:
edge_point(qb_x,qb_y,qb_orientation,max_range)

In [ ]:
receivers = whole_sample.query('position=="WR"|position=="TE"|position == "RB"')

In [ ]:
import math

def calculate_spread(angle, unit='degrees'):
    """
    Calculate the spread (half-angle) of the QB's field of view.

    Parameters:
    - angle: Total field of view angle (degrees or radians).
    - unit: Unit of the angle ('degrees' or 'radians').

    Returns:
    - Spread in radians.
    """
    if unit == 'degrees':
        spread = (angle / 2) * (math.pi / 180)  # Convert to radians
    elif unit == 'radians':
        spread = angle / 2
    else:
        raise ValueError("Unit must be 'degrees' or 'radians'")
    return spread.round(2)


In [ ]:
spread = calculate_spread(qb_orientation)

In [ ]:
whole_sample.query('position=="QB"').o

476113    247.8
Name: o, dtype: float64

In [ ]:
eligibleguys=eligible_receivers(qb_x,qb_y,qb_orientation,calculate_spread(qb_orientation).values[0],
                  max_range,whole_sample.query('position=="WR"|position=="TE"|position == "RB"'))

2557278    83.108121
dtype: float64 2557278    71.635831
dtype: float64 2557278    49.593464
dtype: float64 2557278   -10.635344
dtype: float64
(2557278    38.57
Name: x, dtype: float64, 2557278    26.87
Name: y, dtype: float64) (2557278    83.108121
dtype: float64, 2557278    49.593464
dtype: float64) (2557278    71.635831
dtype: float64, 2557278   -10.635344
dtype: float64)


In [ ]:
# If eligibleguys are indices of the original DataFrame
eligible_receivers = whole_sample.loc[eligibleguys]

In [ ]:
sample_pass=week_1.query('event == "pass_forward"').sample(n=1)

In [ ]:
forwardplay=week_1.query('gameId == @sample_pass.iloc[0].gameId & playId == @sample_pass.iloc[0].playId')

In [ ]:
week_2 =  week_1.copy()

In [ ]:
has_fractional = week_2['time'].str.contains(r'\.\d+$')
week_2.loc[has_fractional, 'time'] = pd.to_datetime(week_2.loc[has_fractional, 'time'], format='%Y-%m-%d %H:%M:%S.%f')

# Parse rows without fractional seconds
week_2.loc[~has_fractional, 'time'] = pd.to_datetime(week_2.loc[~has_fractional, 'time'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
!df -h

In [ ]:
week_2['time_diff']=week_2.groupby(['gameId','playId']).time.diff()

In [ ]:
def pass_speed(df):
    football_df = df[df['displayName']=='football']
    # Sort the DataFrame by gameId, playId, and frameId
    football_df = football_df.sort_values(['gameId', 'playId', 'frameId']).reset_index(drop=True)

    # Identify the ranges of interest
    ranges = []
    for (game_id, play_id), group in football_df.groupby(['gameId', 'playId']):
        in_range = False
        start_frame = None

        for i, row in group.iterrows():
            if row['event'] == 'pass_forward' and not in_range:
                in_range = True
                start_frame = row['frameId']
            elif pd.notna(row['event']) and in_range:
                ranges.append((game_id, play_id, start_frame, row['frameId']))
                in_range = False  # End the range once the next event is reached

    # Create a DataFrame to store the ranges
    ranges_df = pd.DataFrame(ranges, columns=['gameId', 'playId', 'start_frame', 'end_frame'])
    print(ranges_df.shape)
    # Initialize a list to store filtered results
    filtered_frames = []

    # Iterate over each range in ranges_df and filter frames
    for _, range_row in ranges_df.iterrows():
        game_id = range_row['gameId']
        play_id = range_row['playId']
        start_frame = range_row['start_frame']
        end_frame = range_row['end_frame']

        # Filter frames for the current range
        filtered_range = football_df[
            (football_df['gameId'] == game_id) &
            (football_df['playId'] == play_id) &
            (football_df['frameId'] >= start_frame) &
            (football_df['frameId'] <= end_frame)
        ].copy()
        filtered_range.loc[:, 'start_frame'] = start_frame  # Explicitly set 'start_frame'
        filtered_range.loc[:, 'end_frame'] = end_frame      # Explicitly set 'end_frame'
        # Append the filtered range to the list
        filtered_frames.append(filtered_range)

    # Concatenate all filtered frames into a single DataFrame
    filtered_df = pd.concat(filtered_frames, ignore_index=True)
    # print(filtered_df.groupby(['gameId', 'playId', 'start_frame', 'end_frame']).size())

    # Group by gameId, playId, start_frame, and end_frame
    def calculate_distance(group):
        first_frame = group.iloc[0]
        last_frame = group.iloc[-1]
        dx = last_frame['x'] - first_frame['x']
        dy = last_frame['y'] - first_frame['y']
        return np.sqrt(dx**2 + dy**2)

    # Group by gameId, playId, start_frame, and end_frame and apply calculations
    pass_stats = (
        filtered_df.groupby(['gameId', 'playId', 'start_frame', 'end_frame'])
        .apply(lambda group: pd.Series({
            'average_speed': group['s'].mean(),
            'total_distance': calculate_distance(group)
        }))
        .reset_index()
    )

    return pass_stats


In [ ]:
pass_speeds=pass_speed(week_1)


(1078, 4)


<ipython-input-15-64c7a86fbc47>:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: pd.Series({


In [ ]:
pass_speeds

,gameId,playId,start_frame,end_frame,average_speed,total_distance
0,2022090800,56,167,172,11.630000,9.234988
1,2022090800,122,142,148,8.541429,6.323448
2,2022090800,167,121,126,15.585000,11.083127
3,2022090800,212,153,159,17.197143,10.775119
4,2022090800,236,119,134,12.766250,22.477731
...,...,...,...,...,...,...
1073,2022091200,3628,148,154,17.440000,14.672563
1074,2022091200,3723,337,343,15.358572,9.994003
1075,2022091200,3747,61,66,5.943333,2.955757
1076,2022091200,3795,173,174,2.330000,0.214709


In [ ]:
pass_speeds[['average_speed','total_distance']] = pass_speeds[['average_speed','total_distance']].apply(lambda x:x.round(2))

In [ ]:
# Define distance bins (ranges) and labels
bins = [0, 10, 20, 30, 40,50,60,70]  # Adjust the bins as needed
labels = ['0-10', '11-20', '21-30', '31-40','41-50','51-60','61-70']


In [ ]:
# Assign each row to a distance range
pass_speeds['distance_range'] = pd.cut(pass_speeds['total_distance'], bins=bins, labels=labels, right=True)
print(pd.cut(pass_speeds['total_distance'], bins=bins, labels=labels, right=True)
)
# Group by the distance range and calculate the average speed for each range
summary_df = pass_speeds.groupby('distance_range')['average_speed'].mean().reset_index()

# Rename columns for clarity
summary_df.rename(columns={'average_speed': 'average_speed_in_range'}, inplace=True)

print(summary_df)


0        0-10
1        0-10
2       11-20
3       11-20
4       21-30
        ...  
1073    11-20
1074     0-10
1075     0-10
1076     0-10
1077    11-20
Name: total_distance, Length: 1078, dtype: category
Categories (7, object): ['0-10' < '11-20' < '21-30' < '31-40' < '41-50' < '51-60' < '61-70']
  distance_range  average_speed_in_range
0           0-10                9.741576
1          11-20               16.000305
2          21-30               18.201609
3          31-40               18.393800
4          41-50               17.687297
5          51-60               18.557727
6          61-70               19.310000


<ipython-input-19-8d8da8ce420b>:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary_df = pass_speeds.groupby('distance_range')['average_speed'].mean().reset_index()


In [ ]:
summary_df = summary_df.round(2)

In [ ]:
summary_df

,distance_range,average_speed_in_range
0,0-10,9.74
1,11-20,16.00
2,21-30,18.20
3,31-40,18.39
4,41-50,17.69
5,51-60,18.56
6,61-70,19.31


In [ ]:
def select_random_post_snap_pre_pass(df):
    """
    Select a random (gameId, playId, frameId) combination where:
    - The `event` column is NaN.
    - The last valid `event` value was 'ball_snap'.

    Args:
        df (pd.DataFrame): The input DataFrame with columns ['gameId', 'playId', 'frameId', 'event'].

    Returns:
        tuple: A random (gameId, playId, frameId) tuple meeting the constraints.
    """
    # Sort the DataFrame to ensure a meaningful order for checking "last valid value"
    df = df.sort_values(['gameId', 'playId', 'frameId']).reset_index(drop=True)

    # Filter rows where event is NaN
    missing_event_df = df[df['event'].isna()]

    # Identify rows where the last valid event was 'ball_snap'
    def last_event_is_ball_snap(group):
        group['last_valid_event'] = group['event'].ffill()  # Forward-fill events
        return group[(group['event'].isna()) & (group['last_valid_event'] == 'ball_snap')]

    # Apply the filter group-wise for each (gameId, playId)
    valid_rows = (
        df.groupby(['gameId', 'playId'], group_keys=False)
        .apply(last_event_is_ball_snap)
    )

    # Check if any valid rows are available
    if valid_rows.empty:
        raise ValueError("No valid combinations found where event is NaN and last valid event is 'ball_snap'.")

    # Randomly select one row
    random_row = valid_rows.sample(n=1).iloc[0]

    # Return the combination (gameId, playId, frameId)
    return (random_row['gameId'], random_row['playId'], random_row['frameId'])


In [ ]:
def get_eligible_receivers(df,qb_df):
    """
    Perform the combined operation of selecting a random post-snap, pre-pass frame
    and identifying eligible receivers for that frame.

    Args:
        df (pd.DataFrame): The input DataFrame with columns ['gameId', 'playId', 'frameId', 'event', 'nflId', 'x', 'y', 'position'].

    Returns:
        dict: Information about the selected play and eligible receivers.
              Example:
              {
                  'gameId': int,
                  'playId': int,
                  'frameId': int,
                  'eligible_receivers': list of indices
              }
    """
    # Step 1: Select a random (gameId, playId, frameId)
    selected_tuple = select_random_post_snap_pre_pass(df)

    # Step 2: Filter DataFrame for the selected tuple
    filtered_df = df[
        (df['gameId'] == selected_tuple[0]) &
        (df['playId'] == selected_tuple[1]) &
        (df['frameId'] == selected_tuple[2])
    ]

    if filtered_df.empty:
        raise ValueError("No data found for the selected (gameId, playId, frameId).")

    # Step 3: Identify the QB
    qb_row = filtered_df[filtered_df['position'] == 'QB']  # Assuming 'QB' identifies the quarterback
    if qb_row.empty:
        raise ValueError("No QB data found for the selected frame.")

    qb_x, qb_y = qb_row.iloc[0]['x'], qb_row.iloc[0]['y']
    # print(qb_row)
    # Step 4: Determine angle, spread, and max_range
    # These values can be calculated or derived based on game-specific logic.
    # For simplicity, using hardcoded values here, but they could depend on the QB's stats or play data.
    angle = qb_row.iloc[0].o
    spread = calculate_spread(angle)
    qb_name = qb_row.iloc[0].displayName
    max_range = qb_df[qb_df['PLAYER NAME']==qb_name].iloc[0].LCAD # Example: Maximum range of the field of view triangle in yards

    # Step 5: Identify eligible receivers
    receiver_rows = filtered_df.query('position=="WR"|position=="TE"|position == "RB"')  # Assuming 'WR' as receiver position
    eligible = eligible_receivers(qb_x, qb_y, angle, spread, max_range, receiver_rows)

    # Step 6: Return the results
    return {
        'gameId': selected_tuple[0],
        'playId': selected_tuple[1],
        'frameId': selected_tuple[2],
        'eligible_receivers': eligible
    }


In [ ]:
qbstats.drop(columns="Unnamed: 0",inplace=True)

In [ ]:
qbstats.dropna(inplace=True)

In [ ]:
get_eligible_receivers(week_1,qb_stats)

<ipython-input-21-0d6e418281b2>:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(last_event_is_ball_snap)


50 51 60 62


{'gameId': 2022091200,
 'playId': 1028,
 'frameId': 115,
 'eligible_receivers': []}

In [ ]:
airtimes.to_pickle('/kaggle/working/airtimes.pkl')

In [ ]:
airtimes=dfs['airtimes']

In [ ]:
airtimes=airtimes.apply(np.ceil)

In [ ]:
airtimes

,airtime
distance_range,
0-10,5.0
11-20,8.0
21-30,12.0
31-40,17.0
41-50,24.0
51-60,27.0
61-70,30.0


In [ ]:
def process_eligible_receivers(df, qb_df, airtimes):
    """
    Processes eligible receivers and calculates distances and data for the ball-reach frames.

    Args:
        df (pd.DataFrame): Input DataFrame with play and positional data.
        qb_df (pd.DataFrame): QB-specific data containing range information.
        airtimes (pd.DataFrame): DataFrame with average airtime for each distance range.

    Returns:
        pd.DataFrame: Data for eligible receivers including distances and ball-reach frames.
    """
    import numpy as np

    # Helper function to determine the range for a given distance
    def get_airtime(distance):
        for range_str, (lower, upper) in ranges.items():
            if lower <= distance <= upper:
                return airtimes.loc[range_str]

        return None  # Fallback if no range matches

    def range_parser(range_str):
        lower, upper = map(int,range_str.split("-"))
        return lower, upper
    attempt =0
    max_attempts = 20
    result = None
    while not result and attempt<max_attempts:
        # Step 1: Get eligible receivers
        result = get_eligible_receivers(df, qb_df)
        attempt+=1
    # Unpack the results
    game_id, play_id, frame_id, eligible_indices = (
        result["gameId"],
        result["playId"],
        result["frameId"],
        result["eligible_receivers"],
    )
    # print(eligible_indices)

    # Step 2: Filter DataFrame for the selected play and eligible receivers
    filtered_df = df[
        (df["gameId"] == game_id) &
        (df["playId"] == play_id) &
        (df["frameId"] == frame_id)
    ]

    df2 = df[
        (df["gameId"] == game_id) &
        (df["playId"] == play_id)
    ]

    print("eligible_indices:", eligible_indices)
    print("Valid .iloc indices:", list(range(len(filtered_df))))
    if eligible_indices:
        print("not empty")
    else:
        raise ValueError("eligible_indices must not be empty.")
    eligible_receivers = filtered_df.loc[eligible_indices]
    # print(eligible_receivers)

    # Step 3: Get QB position
    qb_row = filtered_df[filtered_df["position"] == "QB"].iloc[0]
    qb_x, qb_y = qb_row["x"], qb_row["y"]
    qb_id = qb_row['nflId']
    print(qb_df.columns)
    qb_details = qb_df[qb_df['PLAYER NAME']==qb_row['displayName']].to_dict(orient='records')
    # Step 4: Calculate distances
    eligible_receivers = eligible_receivers.assign(
        distance_to_qb=lambda row: np.sqrt((row["x"] - qb_x) ** 2 + (row["y"] - qb_y) ** 2)
    )

    # Step 5: Parse ranges and find airtime for each receiver
    ranges = {range_str: range_parser(range_str) for range_str in airtimes.index}
    eligible_receivers = eligible_receivers.drop_duplicates(subset=['nflId'])
    eligible_receivers["airtime"] = eligible_receivers["distance_to_qb"].apply(get_airtime)

    # Step 6: Determine ball-reach frames
    eligible_receivers["ball_reach_frame"] = frame_id + eligible_receivers["airtime"].round().astype(int)
    receiver_ids = eligible_receivers["nflId"].tolist()
    print(eligible_receivers)
    # Step 7: Extract data for ball-reach frames
    ball_reach_df = df[
        (df["gameId"] == game_id) &
        (df["playId"] == play_id) &
        (df["frameId"].isin(eligible_receivers["ball_reach_frame"].unique())) &
        (df["nflId"].isin(receiver_ids))
    ].drop_duplicates(subset=['nflId'])
    # .drop_duplicates(subset=["gameId", "playId", "nflId", "frameId"])

    print(ball_reach_df.duplicated(subset=[ "nflId"]).sum())
    # print(ball_reach_df)
    # Combine and return results
    results = eligible_receivers.merge(
        ball_reach_df,
        on=["gameId", "playId", "nflId"],
        suffixes=("_initial", "_ball_reach")
    )
    # Step 8: Add QB and Closest Defenders
    # Exclude defenders from the QB's team
    qb_team = qb_row["club"]  # Or use "club_initial" if that's the correct column
    defenders = filtered_df[
        (~filtered_df["nflId"].isin(receiver_ids)) &
        (filtered_df["position"] != "QB") &
        (filtered_df['position'] != "football")&
        (filtered_df["club"] != qb_team)
    ]
    rec_defenders = df2[
        (~df2["nflId"].isin(receiver_ids)) &
        (df2['position'] != "football")&
        (df2["club"] != qb_team)
    ]
    closest_def_dict = {}
    for id in receiver_ids:
        reachframe = results.query('nflId == @id')['ball_reach_frame'].iloc[0]
        # Get the receiver's position at the ball reach frame
        x_rec, y_rec = (
             results.query('nflId == @id')['x_ball_reach'].iloc[0],
             results.query('nflId == @id')['y_ball_reach'].iloc[0]
        )
        x_def,y_def = rec_defenders.query('frameId == @reachframe')['x'],rec_defenders.query('frameId == @reachframe')['y']
        rec_defenders = rec_defenders.assign(
            distance_from_rec = lambda row: np.sqrt((x_def -x_rec ) ** 2 + (y_def - y_rec) ** 2)
        )
        closest_rec_def = rec_defenders.nsmallest(4, "distance_from_rec")

        closest_def_dict[id] = closest_rec_def

    # Calculate distances for defenders
    defenders = defenders.assign(
        distance_to_qb=lambda row: np.sqrt((row["x"] - qb_x) ** 2 + (row["y"] - qb_y) ** 2)
    )
    # print(defenders)
    # Select the 3 closest defenders
    closest_defenders = defenders.nsmallest(4, "distance_to_qb")
    qb_dict = {}
    qb_dict[qb_id]  = closest_defenders
    print(closest_defenders)
    qb_row["distance_to_qb"] = 0  # QB is at the origin relative to himself
    all_rows = pd.concat([closest_defenders, pd.DataFrame([qb_row])], ignore_index=True)

    # Add missing columns to defenders to match `results`
    for col in results.columns:
        if col not in all_rows.columns:
            all_rows[col] = np.nan  # Initialize missing columns with NaN

    # Filter for the same initial frame as in `results`
    all_rows = all_rows[all_rows["frameId"] == frame_id]

    def_dict = {f"df{i+1}":df for i,(key,df) in enumerate(closest_def_dict.items())}
    qb_defs = {f"df{i+1}":df for i,(key,df) in enumerate(qb_dict.items())}
    # Combine all rows into the final results
    # results = pd.concat([results, all_rows], ignore_index=True)
    return results, def_dict, qb_defs,qb_details, qb_row

In [ ]:
more_qb_adv = pd.read_csv('/kaggle/input/more-qbstats/FantasyPros_Fantasy_Football_Advanced_Stats_Report_QB.csv')

In [ ]:
save_dfs(more_qb_adv,'/kaggle/working')

In [ ]:
more_qb_adv = dfs['more_qb_adv']

In [ ]:
 pass_options,def_dict,qb_defenders,qb_details, qb_row = process_eligible_receivers(week_1,qb_stats,airtimes)

<ipython-input-21-0d6e418281b2>:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(last_event_is_ball_snap)


118 21 13 2
eligible_indices: [3386992, 3387934]
Valid .iloc indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
not empty
Index(['PLAYER NAME', 'TEAM', 'TT', 'CAY', 'IAY', 'AYD', 'AGG%', 'LCAD',
       'AYTS', 'ATT', 'YDS', 'TD', 'INT', 'RATE', 'COMP%', 'xCOMP%', '+/-'],
      dtype='object')
             gameId  playId    nflId    displayName  frameId   frameType  \
3386992  2022091106     196  44912.0    Jonnu Smith      118  AFTER_SNAP   
3387934  2022091106     196  48097.0  Jakobi Meyers      118  AFTER_SNAP   

                          time  jerseyNumber club playDirection  ...     a  \
3386992  2022-09-11 17:08:53.1          81.0   NE         right  ...  2.40   
3387934  2022-09-11 17:08:53.1          16.0   NE         right  ...  1.21   

          dis       o     dir  event  position  height distance_to_qb airtime  \
3386992  0.08  209.22  132.72    NaN        TE     6-3       8.280368     5.0   
3387934  0.67   71.34   81.85    NaN   

In [ ]:
def_dict

{'df1':              gameId  playId    nflId        displayName  frameId   frameType  \
 3386369  2022091106     196  43321.0     Emmanuel Ogbah      123  AFTER_SNAP   
 3385741  2022091106     196  40030.0       John Jenkins      123  AFTER_SNAP   
 3387625  2022091106     196  47796.0  Christian Wilkins      123  AFTER_SNAP   
 3385584  2022091106     196  38548.0      Melvin Ingram      123  AFTER_SNAP   
 
                           time  jerseyNumber club playDirection  ...      y  \
 3386369  2022-09-11 17:08:53.6          91.0  MIA         right  ...  17.68   
 3385741  2022-09-11 17:08:53.6          77.0  MIA         right  ...  22.55   
 3387625  2022-09-11 17:08:53.6          94.0  MIA         right  ...  25.59   
 3385584  2022-09-11 17:08:53.6           6.0  MIA         right  ...  25.96   
 
             s     a   dis       o     dir  event position height  \
 3386369  1.97  2.21  0.21  318.55  306.37    NaN       DE    6-4   
 3385741  3.72  1.25  0.38  309.10  261.97    

In [ ]:
with open("more_qbstats.pkl", "wb") as f:
    pickle.dump(more_qb_adv, f)


In [ ]:
def calculate_qb_pressure(df, qb_row):
    """
    Calculates pressure on the QB based on the proximity, speed, acceleration,
    and direction of blitzing defenders.

    Parameters:
    - df (DataFrame): Contains defender data with columns:
        ['x', 'y', 's', 'a', 'o', 'dir', 'distance_to_qb', 'position']
    - qb_x (float): QB's x-coordinate.
    - qb_y (float): QB's y-coordinate.
    - qb_orientation (float): QB's orientation in degrees.

    Returns:
    - float: Pressure score on the QB.
    """
    qb_x,qb_y = qb_row['x'],qb_row['y']
    qb_orientation = qb_row['o']
    # Constants for weighting factors
    SPEED_WEIGHT = 2.0
    ACCEL_WEIGHT = 1.5
    PROXIMITY_WEIGHT = 3.0
    ANGLE_WEIGHT = 1.0
    MAX_DISTANCE = 10  # Maximum effective distance for pressure


    # Initialize pressure score
    pressure_score = 0

    for _, defender in df.iterrows():
        # Calculate proximity effect (higher for closer defenders)
        proximity_effect = max(0, 1 - (defender['distance_to_qb'] / MAX_DISTANCE))

        # Calculate angular effect (alignment with QB orientation)
        defender_angle = np.deg2rad(defender['o'])
        qb_orientation_rad = np.deg2rad(qb_orientation)
        angle_diff = np.abs(defender_angle - qb_orientation_rad)
        angle_diff = min(angle_diff, 2 * np.pi - angle_diff)  # Normalize to 0 to π
        angular_effect = 1 - (angle_diff / np.pi)  # High pressure for aligned directions

        # Calculate individual defender's contribution to pressure
        defender_pressure = (
            PROXIMITY_WEIGHT * proximity_effect +
            SPEED_WEIGHT * defender['s'] +
            ACCEL_WEIGHT * defender['a'] +
            ANGLE_WEIGHT * angular_effect
        )

        # Add to total pressure score
        pressure_score += defender_pressure

    return pressure_score


In [ ]:
def calculate_difficulty(qb_row,receiver):
    # Compute target angle
    qb_x,qb_y = qb_row['x'],qb_row['y']
    qb_orientation = qb_row['o']
    receiver_x,receiver_y = receiver['x'],receiver['y']
    target_angle = np.arctan2(receiver_y - qb_y, receiver_x - qb_x)
    distance_to_receiver = receiver['distance_to_qb']
    # Normalize angles to radians
    qb_orientation_rad = np.deg2rad(qb_orientation)
    angle_diff = np.abs(target_angle - qb_orientation_rad)
    angle_diff = min(angle_diff, 2 * np.pi - angle_diff)  # Normalize to 0 to π

    # Calculate angular difficulty multiplier
    k = 2  # Tunable constant
    angular_difficulty = 1 + k * angle_diff

    # Calculate total difficulty
    total_difficulty = distance_to_receiver * angular_difficulty
    return total_difficulty


In [ ]:
import random

def simulate_pass(qb_accuracy, qb_orientation, receiver_position, receiver_speed,
                  defender_proximity, pressure_level):
    # Deterministic components
    distance_factor = 1.0 - pressure_level  # Example scaling
    time_to_ball = 2  # Estimated time (seconds)
    landing_x = receiver_position[0] + (receiver_speed * time_to_ball) + (qb_orientation[0] * distance_factor)
    landing_y = receiver_position[1] + (qb_orientation[1] * distance_factor)

    # Random components
    sigma_x = (1 - qb_accuracy) + (pressure_level * 0.5)
    sigma_y = (1 - qb_accuracy) + (pressure_level * 0.5)
    rand_x = random.gauss(0, sigma_x)
    rand_y = random.gauss(0, sigma_y)

    # Final landing spot
    final_landing_x = landing_x + rand_x
    final_landing_y = landing_y + rand_y

    return final_landing_x, final_landing_y


In [ ]:
def run_pass_simulation(df, qb_df, airtimes, num_simulations=60):
    # Step 1: Process eligible receivers + get data
    results, def_dict, qb_defs, qb_details, qb_row = process_eligible_receivers(df, qb_df, airtimes)

    # Step 2: Calculate QB pressure at initial frame
    qb_defenders_df = list(qb_defs.values())[0]
    qb_x, qb_y = qb_row['x'], qb_row['y']
    qb_defenders_df = qb_defenders_df.assign(
        distance_to_qb=lambda row: np.sqrt((row["x"] - qb_x) ** 2 + (row["y"] - qb_y) ** 2)
    )
    qb_pressure = calculate_qb_pressure(qb_defenders_df, qb_row)

    simulated_passes = []

    for _, receiver_row in results.iterrows():
        receiver_id = receiver_row['nflId']

        # Calculate difficulty score
        receiver_snapshot = {
            'x': receiver_row['x_ball_reach'],
            'y': receiver_row['y_ball_reach'],
            'distance_to_qb': np.sqrt((receiver_row['x_ball_reach'] - qb_x) ** 2 +
                                      (receiver_row['y_ball_reach'] - qb_y) ** 2)
        }
        difficulty_score = calculate_difficulty(qb_row, receiver_snapshot)

        # Receiver details
        receiver_pos = (receiver_row['x_initial'], receiver_row['y_initial'])
        receiver_final_pos = (receiver_row['x_ball_reach'], receiver_row['y_ball_reach'])
        receiver_speed = receiver_row.get('s', 0)

        # Find closest defenders at ball reach frame
        closest_defenders_df = def_dict.get(f"df{list(def_dict.keys()).index(f'df{receiver_id}')+1}", None)
        if closest_defenders_df is None or closest_defenders_df.empty:
            defender_proximity = 10
            closest_defender_pos = None
        else:
            rec_x, rec_y = receiver_final_pos
            closest_defenders_df = closest_defenders_df.assign(
                distance=lambda row: np.sqrt((row["x"] - rec_x) ** 2 + (row["y"] - rec_y) ** 2)
            )
            closest_defender = closest_defenders_df.loc[closest_defenders_df['distance'].idxmin()]
            defender_proximity = closest_defender['distance']
            closest_defender_pos = (closest_defender['x'], closest_defender['y'])

        # QB orientation and accuracy
        qb_orientation_deg = qb_row['o']
        qb_orientation_rad = np.deg2rad(qb_orientation_deg)
        qb_orientation_vec = (np.cos(qb_orientation_rad), np.sin(qb_orientation_rad))
        qb_accuracy = qb_details[0].get('COMP%', 0.7) if qb_details else 0.7
        adjusted_accuracy = qb_accuracy / (1 + 0.05 * difficulty_score)
        pressure_level = min(qb_pressure / 20.0, 1.0)

        # Run multiple simulations
        success_count = 0
        sim_results = []

        for _ in range(num_simulations):
            landing_x, landing_y = simulate_pass(
                qb_accuracy=adjusted_accuracy,
                qb_orientation=qb_orientation_vec,
                receiver_position=receiver_pos,
                receiver_speed=receiver_speed,
                defender_proximity=defender_proximity,
                pressure_level=pressure_level
            )

            # Determine success: Is landing spot closer to receiver or defender?
            dist_to_receiver = np.sqrt((landing_x - receiver_final_pos[0]) ** 2 +
                                       (landing_y - receiver_final_pos[1]) ** 2)
            if closest_defender_pos is None:
                dist_to_defender = float('inf')
            else:
                dist_to_defender = np.sqrt((landing_x - closest_defender_pos[0]) ** 2 +
                                           (landing_y - closest_defender_pos[1]) ** 2)

            success = int(dist_to_receiver < dist_to_defender)
            success_count += success

            sim_results.append({
                "landing_x": landing_x,
                "landing_y": landing_y,
                "success": success
            })

        success_rate = success_count / num_simulations

        simulated_passes.append({
            "receiver_id": receiver_id,
            "success_rate": success_rate,
            "num_simulations": num_simulations,
            "pressure_level": pressure_level,
            "defender_proximity": defender_proximity,
            "qb_pressure": qb_pressure,
            "difficulty_score": difficulty_score,
            "receiver_final_pos": receiver_final_pos,
            "simulations": sim_results  # optional: to inspect raw sim data
        })

    return simulated_passes


In [ ]:
run_pass_simulation(week_1,qb_stats,airtimes)

<ipython-input-21-0d6e418281b2>:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


45 -30 -30 4
eligible_indices: [6718106, 6718306]
Valid .iloc indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
not empty
Index(['PLAYER NAME', 'TEAM', 'TT', 'CAY', 'IAY', 'AYD', 'AGG%', 'LCAD',
       'AYTS', 'ATT', 'YDS', 'TD', 'INT', 'RATE', 'COMP%', 'xCOMP%', '+/-'],
      dtype='object')
             gameId  playId    nflId   displayName  frameId   frameType  \
6718106  2022090800     299  43399.0  Tyler Higbee      155  AFTER_SNAP   
6718306  2022090800     299  44881.0   Cooper Kupp      155  AFTER_SNAP   

                          time  jerseyNumber club playDirection  ...     a  \
6718106  2022-09-09 00:33:31.2          89.0   LA         right  ...  2.47   
6718306  2022-09-09 00:33:31.2          10.0   LA         right  ...  0.31   

          dis      o     dir  event  position  height distance_to_qb airtime  \
6718106  0.03  58.52    5.41    NaN        TE     6-6       4.886686     5.0   
6718306  0.72   1.95  349.59    NaN        

ValueError: 'df43399.0' is not in list

In [ ]:
pass_options.iloc[0]['nflId']